In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load Data
df = pd.read_csv("C:/Users/jay.jain/Desktop/Codes - Jay/Assignment 1/FEIcm1.csv")
df.columns = ['Ticker1', 'Ticker2', 'Description', 'Timestamp', 'Col5', 'Frequency',
              'Open', 'High', 'Low', 'Close', 'Volume']

df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors='coerce')
df = df.dropna(subset=["Timestamp"]).sort_values("Timestamp")
df = df[(df["Timestamp"].dt.year >= 2023) & (df["Timestamp"].dt.year <= 2024)]
df.set_index("Timestamp", inplace=True)

# ------------------------------
# Weekly Resampling
# ------------------------------
weekly = df['Close'].resample('W-FRI').last().dropna()
weekly = weekly.to_frame(name='Close')

# Bollinger Bands (Weekly)
window = 20
weekly['MA'] = weekly['Close'].rolling(window).mean()
weekly['STD'] = weekly['Close'].rolling(window).std()
weekly['Upper'] = weekly['MA'] + 2 * weekly['STD']
weekly['Lower'] = weekly['MA'] - 2 * weekly['STD']
weekly['BBW'] = (weekly['Upper'] - weekly['Lower']) / weekly['MA']

# Re-entry Logic
weekly['OutsideLower'] = weekly['Close'] < weekly['Lower']
weekly['ReentryLong'] = (weekly['Close'] > weekly['Lower']) & (weekly['OutsideLower'].shift(1) == True)

weekly['OutsideUpper'] = weekly['Close'] > weekly['Upper']
weekly['ReentryShort'] = (weekly['Close'] < weekly['Upper']) & (weekly['OutsideUpper'].shift(1) == True)

# ------------------------------
# Plot
# ------------------------------
plt.figure(figsize=(14, 6))
plt.plot(weekly.index, weekly['Close'], label='Weekly Close', color='black')
plt.plot(weekly.index, weekly['Upper'], linestyle='--', color='red', label='Upper Band')
plt.plot(weekly.index, weekly['Lower'], linestyle='--', color='green', label='Lower Band')
plt.fill_between(weekly.index, weekly['Upper'], weekly['Lower'], color='gray', alpha=0.1)

# Re-entry signals
#plt.scatter(weekly[weekly['ReentryLong']].index, weekly[weekly['ReentryLong']]['Close'],
 #           color='blue', marker='^', s=100, label='Re-entry Long')
#plt.scatter(weekly[weekly['ReentryShort']].index, weekly[weekly['ReentryShort']]['Close'],
  #          color='orange', marker='v', s=100, label='Re-entry Short')

plt.title("Weekly Bollinger Bands (2012–2015)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()